In [3]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import random
import pytorch_msssim
import torch
from torch.autograd import Variable
from skimage.measure import block_reduce
import math
import copy
from PIL import Image, ImageFilter
import skimage.measure
from scipy import ndimage
import scipy.signal as signal

In [18]:
##########################################
################ Filters #################
##########################################

In [7]:
# Box Filter

def box_filter(img):
    result = copy.copy(img)
    cv2.blur(img, (2, 2))
    return result

In [8]:
# Gaussian Filter

def gaussian_filter(img, kernel_size):
    result = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    return result

# Gaussian Filter Kernel Size 3

def gaussian_filter_3(img):
    result = cv2.GaussianBlur(img, (3, 3), 0)
    return result

# Gaussian Filter Kernel Size 5

def gaussian_filter_5(img):
    result = cv2.GaussianBlur(img, (5, 5), 0)
    return result

In [9]:
# Lanczos Filter

def lanczos_filter_single_channel(img):
    lanczos_kernel = np.array([[0, -0.032, 0, 0.284, 0.496, 0.284, 0, -0.032, 0]])
    return signal.convolve(signal.convolve(img, lanczos_kernel, mode='same'), lanczos_kernel.T, mode='same')

def lanczos_filter(img):
    # separate the channels
    img_r = img[:,:,0]
    img_g = img[:,:,1]
    img_b = img[:,:,2]
    
    # apply lanczos filter to every channel individually
    result_r = lanczos_filter_single_channel(img_r)
    result_g = lanczos_filter_single_channel(img_g)
    result_b = lanczos_filter_single_channel(img_b)
    
    # stack the channels back
    result = np.dstack((result_r, result_g, result_b))
    return result

In [10]:
# Sinc Filter

def sinc_filter_single_channel(img):
    rows, cols = img.shape
    crow, ccol = int(rows/2), int(cols/2)
    
    # perform a discrete fourier transform
    dft = cv2.dft(np.float32(img), flags = cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    
    # cut off high frequencies
    mask = np.zeros((rows,cols,2),np.uint8)
    mask[crow-30:crow+30, ccol-30:ccol+30] = 1
    fshift = dft_shift * mask
    
    # perform an inverse discrete fourier transform
    f_ishift = np.fft.ifftshift(fshift)
    img_res = cv2.idft(f_ishift, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT)
    
    return img_res

def sinc_filter(img):
    rows, cols, channels = img.shape
    crow,ccol = int(rows/2), int(cols/2)
    
    # separate the channels
    img_r = img[:,:,0]
    img_g = img[:,:,1]
    img_b = img[:,:,2]
    
    # apply sinc filter to every channel individually
    img_res_r = sinc_filter_single_channel(img_r)
    img_res_g = sinc_filter_single_channel(img_g)
    img_res_b = sinc_filter_single_channel(img_b)
    
    # stack the channels back
    result = np.dstack((img_res_r, img_res_g, img_res_b))

    return result

In [24]:
#########################################
########### Subsampling #################
#########################################

In [13]:
# Take every first pixel

def sample_first(img):
    result = img[::2, ::2]
    return result

In [14]:
# Take every second pixel

def sample_second(img):
    result = img[::2, 1::2]
    return result

In [15]:
# Take every third pixel

def sample_third(img):
    result = img[1::2, ::2]
    return result

In [16]:
# Take every fourth pixel

def sample_fourth(img):
    result = img[1::2, 1::2]
    return result

In [17]:
# Take average of four pixels

def sample_average(img):
#     result = (sample_first(img) + sample_second(img) + sample_third(img) + sample_fourth(img)) / 4;
    result = skimage.measure.block_reduce(img, (2,2,1), np.mean)
    return result

In [18]:
# Take max of four pixels

def sample_max(img):
    result = skimage.measure.block_reduce(img, (2,2,1), np.max)
    return result

In [19]:
# Take min of four pixels

def sample_min(img):
    result = skimage.measure.block_reduce(img, (2,2,1), np.min)
    return result

In [20]:
# Take median of four pixels

def sample_median(img):
    result = skimage.measure.block_reduce(img, (2,2,1), np.median)
    return result

In [21]:
# Take random of four pixels

def sample_random(img):
    rows, cols, channels = img.shape
    result = np.zeros((rows // 2, cols // 2, 3))
    rx = 0
    ry = 0
    for i in range(0, rows - 1, 2):
        ry = 0
        for j in range(0, cols - 1, 2):
            x = np.random.choice([i, i + 1])
            y = np.random.choice([j, j + 1])
            result[rx][ry] = img[x][y]
            ry = ry + 1
        rx = rx + 1
    return result

In [23]:
# Take most extreme of four pixels

def get_abs(rgb):
    r = rgb[0]
    g = rgb[1]
    b = rgb[2]
    return r*r + g*g + b*b

def sample_extreme(img):
    rows, cols, channels = img.shape
    result = np.zeros((rows // 2, cols // 2, 3))
    rx = 0
    ry = 0
    for i in range(0, rows - 1, 2):
        ry = 0
        for j in range(0, cols - 1, 2):
            mean = (img[i][j] + img[i][j+1] + img[i+1][j] + img[i+1][j+1]) / 4
            mean_abs = get_abs(mean)
            max_diff = 0
            val = [0, 0, 0]
            
            for x in range(i, i + 2):
                for y in range(j, j + 2):
                    diff = abs(get_abs(img[x][y]) - mean_abs)
                    if diff > max_diff:
                        val = img[x][y]
                        max_diff = diff
            
            result[rx][ry] = val
            ry = ry + 1
        rx = rx + 1
    return result

In [24]:
##########################################

In [25]:
############ Comparison Functions #############

In [26]:
# get SSIM

def get_ssim(original, downscaled):
    scale = original.shape[0] / downscaled.shape[0]
    img1 = torch.from_numpy(np.rollaxis(original, 2)).float().unsqueeze(0)/255.0
    upscaled_from_downscaled = cv2.resize(downscaled, None, fx = scale, fy = scale, interpolation = cv2.INTER_LINEAR)
    img2 = torch.from_numpy(np.rollaxis(upscaled_from_downscaled, 2)).float().unsqueeze(0)/255.0
    return pytorch_msssim.msssim(img1, img2)

In [38]:
#################################################

In [27]:
# List of low pass filters and subsampling methods

low_pass_filters = {
    "box": box_filter,
    "gaussian_3": gaussian_filter_3,
    "gaussian_5": gaussian_filter_5,
    "lanczos": lanczos_filter,
    "sinc": sinc_filter
#     "mlaagic": magic_filter
}

subsampling_methods = {
    "first": sample_first,
    "second": sample_second,
    "third": sample_third,
    "fourth": sample_fourth,
    "average": sample_average,
    "max": sample_max,
    "min": sample_min,
    "median": sample_median,
    "random": sample_random,
    "extreme": sample_extreme
}

In [36]:
# constants

# folder containing 256x256 images
IMAGE_DIR = 'C:/Users/sukan/Documents/TUM/Courses/Winter 2019/Thesis/Westermann - Texture Map/threejsrockstutorial/mipmap_LPF_SS'
INPUT_SIZE = 256

In [29]:
# derived constants

INPUT_DIR_NAME = str(INPUT_SIZE) + 'x' + str(INPUT_SIZE)
IMAGE_INPUT_DIR = os.path.join(IMAGE_DIR, INPUT_DIR_NAME)

In [30]:
# process files

file_names = os.listdir(IMAGE_INPUT_DIR)
images = []
cnt = 0
print(str(len(file_names)) + " files found.")
for file_name in file_names:
    images.append({
        "name": file_name,
        "image": cv2.imread(os.path.join(IMAGE_INPUT_DIR, file_name))
    })
    cnt = cnt + 1
    if (cnt % 100 == 0):
        print('read ' + str(cnt) + ' images.')

201 files found.
read 100 images.
read 200 images.


In [37]:
# downscale image files - generate mipmaps

curr_size = INPUT_SIZE
first_input_folder = True

while (curr_size >= 2):
    print ("input size: " + str(curr_size))
    curr_output_size = int(curr_size / 2)
    curr_input_folder_name = str(curr_size) + 'x' + str(curr_size)
    curr_output_folder_name = str(curr_output_size) + 'x' + str(curr_output_size)
    curr_input_folder = os.path.join(IMAGE_DIR, curr_input_folder_name)
    curr_output_folder = os.path.join(IMAGE_DIR, curr_output_folder_name, 'lpf_ss')
    
    read all images in current input folder
    file_names = os.listdir(curr_input_folder)
    
    # loop through all lpfs
    for lpf in low_pass_filters:
        print("lpf: " + lpf)
        # loop through all subsampling methods
        for ss in subsampling_methods:
            print("ss: " + ss)
            lpf_ss_folder = lpf + '_' + ss
            
            if (first_input_folder):
                curr_lpf_ss_input_folder = curr_input_folder
            else:
                curr_lpf_ss_input_folder = os.path.join(curr_input_folder, 'lpf_ss', lpf_ss_folder)
                file_names = os.listdir(curr_lpf_ss_input_folder)
            
            curr_lpf_ss_output_folder = os.path.join(curr_output_folder, lpf_ss_folder)
            
            # if images for this lpf and ss and size have been generated move on to next ss
            if not (os.path.exists(curr_lpf_ss_output_folder)):
                os.mkdir(curr_lpf_ss_output_folder)
            
            c = 0
            for file_name in file_names:
                image = cv2.imread(os.path.join(curr_lpf_ss_input_folder, file_name))
                output_image_path = os.path.join(curr_lpf_ss_output_folder, file_name)
                
                if (os.path.exists(output_image_path)):
                    continue
                
                c = c + 1
                lpf_image = low_pass_filters[lpf](image)
                downscaled = subsampling_methods[ss](lpf_image).astype(np.uint8)
                cv2.imwrite(output_image_path, downscaled)
#             print (str(c) + " images downscaled now.")
    
    curr_size = int(curr_size / 2)
    first_input_folder = False

input size: 256
lpf: box
ss: first
4 images downscaled now.
ss: second
4 images downscaled now.
ss: third
4 images downscaled now.
ss: fourth
4 images downscaled now.
ss: average
4 images downscaled now.
ss: max
4 images downscaled now.
ss: min
4 images downscaled now.
ss: median
4 images downscaled now.
ss: random
4 images downscaled now.
ss: extreme


C:\Users\sukan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in ubyte_scalars
  import sys


4 images downscaled now.
lpf: gaussian_3
ss: first
4 images downscaled now.
ss: second
4 images downscaled now.
ss: third
4 images downscaled now.
ss: fourth
4 images downscaled now.
ss: average
4 images downscaled now.
ss: max
4 images downscaled now.
ss: min
4 images downscaled now.
ss: median
4 images downscaled now.
ss: random
4 images downscaled now.
ss: extreme
4 images downscaled now.
lpf: gaussian_5
ss: first
4 images downscaled now.
ss: second
4 images downscaled now.
ss: third
4 images downscaled now.
ss: fourth
4 images downscaled now.
ss: average
4 images downscaled now.
ss: max
4 images downscaled now.
ss: min
4 images downscaled now.
ss: median
4 images downscaled now.
ss: random
4 images downscaled now.
ss: extreme
4 images downscaled now.
lpf: lanczos
ss: first
4 images downscaled now.
ss: second
4 images downscaled now.
ss: third
4 images downscaled now.
ss: fourth
4 images downscaled now.
ss: average
4 images downscaled now.
ss: max
4 images downscaled now.
ss: min
4 

C:\Users\sukan\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log


4 images downscaled now.
ss: fourth
4 images downscaled now.
ss: average
4 images downscaled now.
ss: max
4 images downscaled now.
ss: min


C:\Users\sukan\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log


4 images downscaled now.
ss: median
4 images downscaled now.
ss: random
4 images downscaled now.
ss: extreme
4 images downscaled now.
input size: 64
lpf: box
ss: first
4 images downscaled now.
ss: second
4 images downscaled now.
ss: third
4 images downscaled now.
ss: fourth
4 images downscaled now.
ss: average
4 images downscaled now.
ss: max
4 images downscaled now.
ss: min
4 images downscaled now.
ss: median
4 images downscaled now.
ss: random
4 images downscaled now.
ss: extreme
4 images downscaled now.
lpf: gaussian_3
ss: first
4 images downscaled now.
ss: second
4 images downscaled now.
ss: third
4 images downscaled now.
ss: fourth
4 images downscaled now.
ss: average
4 images downscaled now.
ss: max
4 images downscaled now.
ss: min
4 images downscaled now.
ss: median
4 images downscaled now.
ss: random
4 images downscaled now.
ss: extreme
4 images downscaled now.
lpf: gaussian_5
ss: first
4 images downscaled now.
ss: second
4 images downscaled now.
ss: third
4 images downscaled n

C:\Users\sukan\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log


4 images downscaled now.
ss: second
4 images downscaled now.
ss: third
4 images downscaled now.
ss: fourth
4 images downscaled now.
ss: average
4 images downscaled now.
ss: max
4 images downscaled now.
ss: min
4 images downscaled now.
ss: median
4 images downscaled now.
ss: random
4 images downscaled now.
ss: extreme
4 images downscaled now.
input size: 8
lpf: box
ss: first
4 images downscaled now.
ss: second
4 images downscaled now.
ss: third
4 images downscaled now.
ss: fourth
4 images downscaled now.
ss: average
4 images downscaled now.
ss: max
4 images downscaled now.
ss: min
4 images downscaled now.
ss: median
4 images downscaled now.
ss: random
4 images downscaled now.
ss: extreme
4 images downscaled now.
lpf: gaussian_3
ss: first
4 images downscaled now.
ss: second
4 images downscaled now.
ss: third
4 images downscaled now.
ss: fourth
4 images downscaled now.
ss: average
4 images downscaled now.
ss: max
4 images downscaled now.
ss: min
4 images downscaled now.
ss: median
4 image